In [9]:
import pandas as pd 
import numpy as np 
import plotly.express as px 
import petpy 
import pickle
import sqlite3
import time
import datetime

In [10]:
with open('/home/malcolm/credentials/petfinder_credentials.pkl', 'rb') as hnd:
    creds = pickle.load(hnd)

## Create API Request yourself

In [11]:
import urllib
import requests

In [12]:
endpoint = 'oauth2/token'
host = 'http://api.petfinder.com/v2/'
url = urllib.parse.urljoin(host, endpoint)

data = {
    'grant_type': 'client_credentials',
    'client_id': creds['key'],
    'client_secret': creds['secret']
}

r = requests.post(url, data=data)
api_key = r.json()['access_token']

In [13]:
animals_endpt = 'https://api.petfinder.com/v2/animals/'
params = dict(type='dog', 
    location='10475', distance=30,
    limit=100, page=1
             )
headers = {'Authorization' : 'Bearer ' + api_key}
# headers = {'Authorization' : 'Bearer ' + pf_key}

In [14]:
results = requests.get(animals_endpt
                       , params=params
                       , headers=headers)
results

<Response [429]>

In [15]:
dogs_rslts_json = results.json()
dogs_rslts_json.keys()

dict_keys(['type', 'status', 'title', 'detail'])

In [16]:
n_dog_pages = dogs_rslts_json['pagination']['total_pages']
n_dog_pages

KeyError: 'pagination'

In [ ]:
params['type'] = 'cat'
results = requests.get(animals_endpt
                       , params=params
                       , headers=headers)
assert(results.status_code == 200), f"Status code not 200: It's {results.status_code}"
cats_rslts_json = results.json()
n_cat_pages = cats_rslts_json['pagination']['total_pages']
n_cat_pages

## Get All Animals For Region


In [ ]:
pf = petpy.Petfinder(key=creds['key'], secret=creds['secret'])

In [ ]:
params.keys()

In [ ]:
petpy_dict = params.copy()
# petpy_dict['animal_type'] = petpy_dict.pop('type')
petpy_dict['animal_type'] = 'dog'
petpy_dict['results_per_page'] = petpy_dict.pop('limit')
petpy_dict['pages'] = n_dog_pages
petpy_dict['return_df'] = True
del(petpy_dict['page'])
if 'type' in petpy_dict.keys():
    del(petpy_dict['type'])
print(petpy_dict)

In [ ]:
adoptable_dogs = pf.animals(**petpy_dict)
adoptable_dogs['date_saved'] = str(datetime.datetime.now().date())
adoptable_dogs

## Cats

In [ ]:
time.sleep(5)
petpy_dict['animal_type'] = 'cat'
petpy_dict['pages'] = n_cat_pages
adoptable_cats = pf.animals(**petpy_dict)
adoptable_cats['date_saved'] = str(datetime.datetime.now().date())
adoptable_cats

## Organizations

In [ ]:
all_orgs = pd.concat([adoptable_dogs['organization_id'].iloc[:,0], 
            adoptable_cats['organization_id'].iloc[:,0]]).unique()
all_orgs

In [ ]:
all_orgs_list = list(all_orgs)
len(all_orgs_list)

In [ ]:
orgs = pf.organizations(all_orgs_list)
orgs

In [ ]:
len(orgs['organizations'])

## Export to SQL

In [ ]:
# Prep dataset for export into SQL 
adoptable_dogs = adoptable_dogs.loc[:,~adoptable_dogs.columns.duplicated()]
adoptable_cats = adoptable_cats.loc[:,~adoptable_cats.columns.duplicated()]
for x in ['tags', 'photos', 'videos']:
    adoptable_dogs[x] = str(adoptable_dogs[x])
    adoptable_cats[x] = str(adoptable_cats[x])

In [ ]:
con = sqlite3.connect('/home/malcolm/petfinder/data/petfinder_qa.db')
cursor = con.cursor()

In [ ]:
table_name_suffix = f"{petpy_dict['location']}_{petpy_dict['distance']}mi"
table_name_suffix



In [ ]:
adoptable_dogs.to_sql('dogs_'+table_name_suffix, con, index=False, if_exists='append')
adoptable_cats.to_sql('cats_'+table_name_suffix, con, index=False, if_exists='append')

## Check Organization's API

In [ ]:
dog_org_ids = adoptable_dogs['organization_id'].unique().tolist()
cat_org_ids = adoptable_cats['organization_id'].unique().tolist()
org_ids_all = set(dog_org_ids + cat_org_ids)
org_ids = list(org_ids_all)
print(len(org_ids))

In [ ]:
org_ids

In [ ]:
orgs_df = pf.organizations(org_ids, return_df =True)
orgs_df['photos'] = str(orgs_df['photos'])
orgs_df['date_saved'] = str(datetime.datetime.now().date())
orgs_df

In [ ]:
orgs_df.to_sql('Organizations', con, index=False, if_exists='replace')

## Graveyard 

In [ ]:
pf = petpy.Petfinder(key=creds['key'], secret=creds['secret'])

In [ ]:
# temp_dogs = pf.animals(animal_type='dog', 
#     location=10475, distance=50,
#     results_per_page=100, pages=1, return_df=False)
temp_dogs.keys()

In [ ]:
adoptable_dogs = pf.animals(animal_type='dog', 
    location=10475, distance=25,
    results_per_page=100, pages=2, return_df=True)
adoptable_dogs

In [ ]:
adoptable_dogs.columns

In [ ]:
vc = adoptable_dogs['contact.address.city'].value_counts()
print(vc.shape)
vc.head(10)

In [ ]:
vc.sum()

In [ ]:
adoptable_dogs['status'].value_counts()

In [ ]:
con.commit()
con.close()